In [45]:
import collections
import requests
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
import transformers
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
import pandas as pd
import xml.etree.ElementTree as ET
import requests
import re
import collections
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
# Fetch XML data from the provided link
response = requests.get("https://news.google.com/rss/search?q=green%20hydrogen&hl=en-IN&gl=IN&ceid=IN:en")

# Parse XML data
root = ET.fromstring(response.text)

# Initialize lists to store data
titles = []
links = []
param_links = []
descriptions = []
dates = []

# Iterate through each item in the XML
for item in root.findall('.//item'):
    # Get title
    title = item.find('title').text

    # Check if the title contains "green hydrogen" in any case
    if "green hydrogen" in title.lower():
        # Get link
        link = item.find('link').text

        # Get parameterized link
        param_link = item.find('guid').text

        # Get description
        description = item.find('description').text


        # Remove anchor tag content from description
        description = re.sub(r'<a.*?>', '', description)
        description = re.sub(r'</a*?>', '', description)
        description = re.sub(r'<font.*?>.*?</font>', '', description)  # Remove content within font tags
        description = re.sub(r'&amp;', '&', description)  # Replace &amp; with &
        description = re.sub(r'&\w+?;', '', description)  # Remove other HTML entities

        # Get date
        date = item.find('pubDate').text

        # Append data to lists
        titles.append(title)
        links.append(link)
        param_links.append(param_link)
        descriptions.append(description)
        dates.append(date)

# Create DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Link': links,
    'Param_Link': param_links,
    'Description': descriptions,
    'Date': dates
})

# Save DataFrame as CSV
df.to_csv('output.csv', index=False)

In [47]:
# %%
# Display DataFrame
print("DataFrame saved as output.csv")
print(df)

df['Description_title'] = df['Title'] + ' - ' + df['Description']

# Print the updated DataFrame
print(df)
df['Description_title'][0]


DataFrame saved as output.csv
                                                Title  \
0   A scalable photoelectrochemical system for gre...   
1   L&T, Matrix chosen for green hydrogen scheme a...   
2   Hero Future Energies to launch new green hydro...   
3   ACME Group, IGX sign MoU to boost green hydrog...   
4   IGX, Acme to develop green hydrogen, ammonia m...   
..                                                ...   
80  Commercial production of green hydrogen - The ...   
81  The cost of Green Hydrogen: What needs to happ...   
82  Green hydrogen production from Indian power gr...   
83  Jakson Green to ship first green hydrogen elec...   
84  India targets 5 MMTPA green hydrogen to curb $...   

                                                 Link  \
0   https://news.google.com/rss/articles/CBMiYGh0d...   
1   https://news.google.com/rss/articles/CBMifmh0d...   
2   https://news.google.com/rss/articles/CBMikAFod...   
3   https://news.google.com/rss/articles/CBMijAFod...   


'A scalable photoelectrochemical system for green hydrogen production - Tech Xplore - A scalable photoelectrochemical system for green hydrogen production'

In [48]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.eval()  # Set model to evaluation mode




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [50]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset


In [51]:
# Tokenize the descriptions and convert them to IDs
tokenized_texts = [tokenizer.encode(description, add_special_tokens=True, max_length=128, truncation=True) for description in df['Description_title']]

# Set maximum sequence length
max_length = 128

# Pad or truncate tokenized sequences to max_length
padded_tokenized_texts = [text[:max_length] + [0] * (max_length - len(text[:max_length])) for text in tokenized_texts]

# Create attention masks
attention_masks = [[int(token_id > 0) for token_id in text] for text in padded_tokenized_texts]

# Convert tokenized input into torch tensors
input_ids = torch.tensor(padded_tokenized_texts)
attention_masks = torch.tensor(attention_masks)

# Create DataLoader
batch_size = 32
data = TensorDataset(input_ids, attention_masks)
sampler = SequentialSampler(data)
dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

# Predict sentiment for each batch
sentiments = []
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]
    softmax_output = torch.softmax(logits, dim=1)
    probabilities, predicted_labels = torch.max(softmax_output, dim=1)
    sentiments.extend(predicted_labels.cpu().numpy())

# Map sentiment labels to 'positive' and 'negative'
sentiment_labels = ['positive' if label == 1 else 'negative' for label in sentiments]

# Add sentiment labels to DataFrame
df['Sentiment'] = sentiment_labels



In [52]:
df['Sentiment']

0     positive
1     positive
2     positive
3     positive
4     positive
        ...   
80    positive
81    positive
82    positive
83    positive
84    positive
Name: Sentiment, Length: 85, dtype: object

In [53]:
df

,Title,Link,Param_Link,Description,Date,Description_title,Sentiment
0,A scalable photoelectrochemical system for gre...,https://news.google.com/rss/articles/CBMiYGh0d...,CBMiYGh0dHBzOi8vdGVjaHhwbG9yZS5jb20vbmV3cy8yMD...,A scalable photoelectrochemical system for gre...,"Sat, 10 Feb 2024 14:00:02 GMT",A scalable photoelectrochemical system for gre...,positive
1,"L&T, Matrix chosen for green hydrogen scheme a...",https://news.google.com/rss/articles/CBMifmh0d...,CBMifmh0dHBzOi8vd3d3LmxpdmVtaW50LmNvbS9jb21wYW...,"L&T, Matrix chosen for green hydrogen scheme a...","Fri, 09 Feb 2024 06:17:12 GMT","L&T, Matrix chosen for green hydrogen scheme a...",positive
2,Hero Future Energies to launch new green hydro...,https://news.google.com/rss/articles/CBMikAFod...,CBMikAFodHRwczovL2VuZXJneS5lY29ub21pY3RpbWVzLm...,Hero Future Energies to launch new green hydro...,"Mon, 12 Feb 2024 02:41:29 GMT",Hero Future Energies to launch new green hydro...,positive
3,"ACME Group, IGX sign MoU to boost green hydrog...",https://news.google.com/rss/articles/CBMijAFod...,CBMijAFodHRwczovL2VuZXJneS5lY29ub21pY3RpbWVzLm...,"ACME Group, IGX sign MoU to boost green hydrog...","Sat, 10 Feb 2024 03:18:57 GMT","ACME Group, IGX sign MoU to boost green hydrog...",positive
4,"IGX, Acme to develop green hydrogen, ammonia m...",https://news.google.com/rss/articles/CBMiggFod...,CBMiggFodHRwczovL3d3dy5saXZlbWludC5jb20vaW5kdX...,"IGX, Acme to develop green hydrogen, ammonia m...","Fri, 09 Feb 2024 13:40:44 GMT","IGX, Acme to develop green hydrogen, ammonia m...",positive
...,...,...,...,...,...,...,...
80,Commercial production of green hydrogen - The ...,https://news.google.com/rss/articles/CBMiWWh0d...,CBMiWWh0dHBzOi8vd3d3LnNlbnRpbmVsYXNzYW0uY29tL2...,Commercial production of green hydrogen,"Wed, 07 Feb 2024 05:31:29 GMT",Commercial production of green hydrogen - The ...,positive
81,The cost of Green Hydrogen: What needs to happ...,https://news.google.com/rss/articles/CBMijQFod...,CBMijQFodHRwczovL2VuZXJneS5lY29ub21pY3RpbWVzLm...,The cost of Green Hydrogen: What needs to happ...,"Mon, 06 Nov 2023 08:00:00 GMT",The cost of Green Hydrogen: What needs to happ...,positive
82,Green hydrogen production from Indian power gr...,https://news.google.com/rss/articles/CBMitwFod...,CBMitwFodHRwczovL2VuZXJneS5lY29ub21pY3RpbWVzLm...,Green hydrogen production from Indian power gr...,"Mon, 06 Nov 2023 08:00:00 GMT",Green hydrogen production from Indian power gr...,positive
83,Jakson Green to ship first green hydrogen elec...,https://news.google.com/rss/articles/CBMiiQFod...,CBMiiQFodHRwczovL2VuZXJneS5lY29ub21pY3RpbWVzLm...,Jakson Green to ship first green hydrogen elec...,"Mon, 01 Jan 2024 08:00:00 GMT",Jakson Green to ship first green hydrogen elec...,positive


In [56]:

# Export the DataFrame to a CSV file
df.to_csv("/content/drive/MyDrive/output_of_google_news.csv", index=False)